In [23]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

from tensorflow.keras.layers import Dense
from tensorflow.keras import Sequential
from tensorflow.keras.callbacks import EarlyStopping

# early stopping

- 과적합을 방지하기 위해, 어느정도 학습이 되면 자동으로 학습을 정지시키는 것.
- 학습횟수를 최대한 늘리고, 학습이 완료되면 학습을 종료시킨다.

## 방법

1) 학습 횟수는 최대한 많이
2) patience : 기다리는 횟수
3) mindelt: cost값의 차이를 보는 것

- tf는 직접 만들 수 있고, keras는 이미 만들어져 있다.

In [2]:
df = pd.read_csv('~/dropbox/python숙달/data/depp/test.csv',header=None)
df.columns = ['q1','q2','mid','final']
df

,q1,q2,mid,final
0,73,80,75,152
1,93,88,93,185
2,89,91,90,180
3,96,98,100,196
4,73,66,70,142
5,53,46,55,101
6,69,74,77,149
7,47,56,60,115
8,87,79,90,175
9,79,70,88,164


In [3]:
x_data = df.iloc[:,:-1].values
y_data = df.iloc[:,[-1]].values

In [4]:
x_data

array([[ 73,  80,  75],
       [ 93,  88,  93],
       [ 89,  91,  90],
       [ 96,  98, 100],
       [ 73,  66,  70],
       [ 53,  46,  55],
       [ 69,  74,  77],
       [ 47,  56,  60],
       [ 87,  79,  90],
       [ 79,  70,  88],
       [ 69,  70,  73],
       [ 70,  65,  74],
       [ 93,  95,  91],
       [ 79,  80,  73],
       [ 70,  73,  78],
       [ 93,  89,  96],
       [ 78,  75,  68],
       [ 81,  90,  93],
       [ 88,  92,  86],
       [ 78,  83,  77],
       [ 82,  86,  90],
       [ 86,  82,  89],
       [ 78,  83,  85],
       [ 76,  83,  71],
       [ 96,  93,  95]], dtype=int64)

# TF로 구현해 보기

In [7]:
x = tf.constant(x_data, tf.float32)
y = tf.constant(y_data, tf.float32)

In [8]:
w = tf.Variable(tf.random.uniform([3,1] ) )
b = tf.Variable(tf.random.uniform([1]))

In [10]:
def compute_loss():
    hx = tf.matmul(x, w) + b
    cost = tf.reduce_mean(tf.square(y - hx))
    return cost

In [20]:
optimizer = Adam(0.1)
patience = 20
pcnt = 0
hist_cost = []
min_delta = 0.001
## 20번동안 앞뒤의 차이가 0.01보다 적으면 중지한다.
for epoch in range(10000):
    optimizer.minimize(compute_loss, var_list=[w,b])
    c = compute_loss().numpy()
    hist_cost.append( c )
    print(epoch, "cost = ", c)
    if epoch > 0:
        if hist_cost[epoch - 1] - hist_cost[epoch] > min_delta:
            pcnt = 0
        else:
            pcnt += 1
        if pcnt > patience:
            print("early stopping")
            break
        else:
            continue

0 cost =  72.866585
1 cost =  105.16513
2 cost =  52.524506
3 cost =  10.528522
4 cost =  64.559494
5 cost =  42.451584
6 cost =  6.4106197
7 cost =  28.600576
8 cost =  43.54614
9 cost =  18.616472
10 cost =  6.9418654
11 cost =  24.678377
12 cost =  29.168896
13 cost =  12.337306
14 cost =  7.208989
15 cost =  19.116686
16 cost =  20.998442
17 cost =  9.759063
18 cost =  7.0934095
19 cost =  15.155935
20 cost =  15.824786
21 cost =  8.16973
22 cost =  7.073345
23 cost =  12.619463
24 cost =  12.295419
25 cost =  7.0591555
26 cost =  7.210329
27 cost =  10.942341
28 cost =  9.779535
29 cost =  6.385029
30 cost =  7.450559
31 cost =  9.66225
32 cost =  7.9730506
33 cost =  6.165438
34 cost =  7.666602
35 cost =  8.505937
36 cost =  6.784356
37 cost =  6.337286
38 cost =  7.677168
39 cost =  7.4234424
40 cost =  6.2207885
41 cost =  6.6859956
42 cost =  7.355842
43 cost =  6.5749965
44 cost =  6.2099414
45 cost =  6.8911533
46 cost =  6.788483
47 cost =  6.1768465
48 cost =  6.460116
49

# keras version

- 훨씬 더 쉽게 수렴한다.

In [35]:
dense = Dense( units = 1, input_dim = 3) # init이 random이지만 xavier나 he를 통해서 init값을 주기에 더 빨리 수렴한다.
model = Sequential([dense])
model.compile(loss = 'mse', optimizer= Adam(0.1))
earlyStop = EarlyStopping(monitor = 'loss', min_delta=0.01, patience = 20)
h = model.fit(x_data, y_data, epochs = 100000, callbacks = [earlyStop])

Epoch 1/100000
1/1 [==============================] - 0s 999us/step - loss: 78409.0156
Epoch 2/100000
1/1 [==============================] - 0s 1ms/step - loss: 65372.6797
Epoch 3/100000
1/1 [==============================] - 0s 1ms/step - loss: 53559.8789
Epoch 4/100000
1/1 [==============================] - 0s 0s/step - loss: 42986.5508
Epoch 5/100000
1/1 [==============================] - 0s 1ms/step - loss: 33659.9297
Epoch 6/100000
1/1 [==============================] - 0s 1ms/step - loss: 25576.1719
Epoch 7/100000
1/1 [==============================] - 0s 1ms/step - loss: 18718.4629
Epoch 8/100000
1/1 [==============================] - 0s 1000us/step - loss: 13054.8438
Epoch 9/100000
1/1 [==============================] - 0s 0s/step - loss: 8536.0361
Epoch 10/100000
1/1 [==============================] - 0s 0s/step - loss: 5093.8667
Epoch 11/100000
1/1 [==============================] - 0s 1ms/step - loss: 2640.3574
Epoch 12/100000
1/1 [==============================] - 0s 827us/